# Few-shot prompt templates

In [27]:
!pip install --upgrade --quiet langchain langchain-openai

In [28]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*****************************************************"

## Using an example set

### Create the example set

In [50]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

### Create a formatter for the few-shot examples

In [51]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

In [52]:
print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



### Feed examples and formatter to FewShotPromptTemplate

In [53]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

In [54]:
print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

## Using an example selector

### Feed examples into ExampleSelector

In [59]:
!pip install --upgrade --quiet chromadb

In [60]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

question: Who was the maternal grandfather of George Washington?


### Feed example selector into FewShotPromptTemplate

In [61]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


## FewShotChatMessagePromptTemplate

In [62]:
examples = [
    {
        "instruction": "Please write the meeting minutes",
         "input": "On December 25, 2023, the marketing strategy meeting of Company The main objective was to establish a marketing strategy and discuss ideas for new social media campaigns for the first half of 2024. Team leader Jiwon Shin provided a brief overview of recent market trends, followed by each team member discussing strategic ideas in their field. announced.",
         "answer": """
Meeting minutes: XYZ company marketing strategy meeting
Date: December 25, 2023
Location: XYZ company conference room
Attendees: Jiwon Shin (Marketing Team Leader), Hasu Lee (Digital Marketer), Duck Kim (Social Media Manager)

1. Opening
    - The meeting began with an opening speech by Team Leader Jiwon Shin.
    - The purpose of the meeting is to establish a marketing strategy for the first half of 2024 and discuss new social media campaign ideas.

2. Market trend overview (Kim Su-jin)
    - Team leader Jiwon Shin presented analysis of recent market trends.
    - Share insights on consumer behavior changes and competitor strategies.

3. Digital Marketing Strategy (Hasu Lee)
    - Hasu Lee gave a presentation on digital marketing strategy.
    - Focuses on online advertising and SEO optimization measures.

4. Social media campaign (Duck Kim)
    - Duck Kim proposed an idea for a new social media campaign.
    - Describes plans for influencer marketing and content strategy.

5. Comprehensive discussion
    - Sharing and discussing ideas among team members.
    - Discuss budget and resource allocation for each strategy.

6. Finishing
    - Confirm the date and time of the next meeting.
    - Hasu Lee is in charge of organizing and distributing meeting minutes.
""",
     },
     {
         "instruction": "Please summarize the document",
         "input": "This is a 20-page report on the 'Strategy for Sustainable Urban Development'. The report addresses the importance of sustainable urban development, the challenges of current urbanization, and various approaches to make urban development sustainable. It provides a comprehensive coverage of the strategy. The report also highlights examples of successful sustainable urban development in a number of countries and summarizes the lessons learned from these examples.",
         "answer": """
Document Summary: Strategy Report for Sustainable Urban Development

- Importance: Highlights why sustainable urban development is essential and its social, economic and environmental benefits.
- Current problems: Analyze major problems arising from the current urbanization process, such as environmental pollution, resource depletion, and increased inequality.
- Strategy: Presenting various strategies to achieve sustainable urban development. This includes green building, improved public transportation, increased energy efficiency, and increased community involvement.
- Case studies: Presenting examples of successful sustainable development in cities around the world. For example, feasible strategies are explained through examples such as Copenhagen in Denmark and Yokohama in Japan.
- Lessons learned: Summarizing the key lessons learned from these cases. Lessons highlighted include the importance of a multi-faceted approach, collaboration with the community, and the need for long-term planning.

This report provides an in-depth analysis of how sustainable urban development can be achieved in realistic and effective forms.
""",
     },
     {
         "instruction": "Please correct my sentence",
         "input": "Our company is planning to introduce a new marketing strategy. Through this, communication with customers will be more effective.",
         "answer": "By introducing a new marketing strategy, the company is expected to be able to improve communication with customers more effectively.",
     },
]

In [63]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [64]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [66]:
llm = ChatOpenAI(
    temperature=0,
    max_tokens=2048,
    model_name="gpt-3.5-turbo",
)

In [67]:
chain = final_prompt | llm

response = chain.invoke(
    {
        "instruction": "회의록을 작성해 주세요",
        "input": "2024년 3월 13일, xxx 기술 회사의 제품 개발 팀은 새로운 Langchain 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 김환덕, 주요 개발자인 신지원, UI/UX 디자이너인 박수열이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
    }
)

print(response.content)

회의록

일시: 2024년 3월 13일
장소: xxx 기술 회사 회의실
참석자: 김환덕 (프로젝트 매니저), 신지원 (주요 개발자), 박수열 (UI/UX 디자이너)

1. 회의 개최
- 김환덕 프로젝트 매니저가 회의를 주관하였다.
- 새로운 Langchain 프로젝트에 대한 주간 진행 상황을 검토하기 위한 회의임을 안내하였다.

2. 현재 진행 상황 검토
- 각 참석자는 자신의 작업 영역에 대한 업데이트를 제공하였다.
- 프로젝트의 현재 진행 상황을 종합적으로 검토하였다.

3. 다가오는 마일스톤 계획 수립
- 팀은 다가오는 마일스톤에 대한 계획을 수립하기 위해 토론하였다.
- 다음 주까지의 목표를 설정하고 업무 일정을 조정하였다.

4. 향후 일정
- 다음 회의 일정 및 안건을 확인하고 마무리하였다.
- 박수열 UI/UX 디자이너가 회의록을 작성하여 배포할 예정임을 안내하였다.

이상으로 회의록을 마칩니다.


In [68]:
response = chain.invoke(
    {
        "instruction": "문서를 요약해 주세요",
        "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
    }
)

print(response.content)

이 보고서는 '2023년 글로벌 경제 전망'에 대한 상세한 30페이지의 보고서로, 세계 경제의 현재 상황, 주요 국가의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해의 경제 예측을 다룹니다. 또한, 다양한 경제적, 정치적, 환경적 요인이 세계 경제에 미치는 영향을 분석하고 있습니다.


In [69]:
response = chain.invoke(
    {
        "instruction": "문장을 교정해 주세요",
        "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
    }
)

print(response.content)

회사는 올해 매출이 증가할 것으로 예상하며, 새로운 전략이 잘 작동하고 있는 것으로 판단된다.
